In [1]:
# Import necessary libraries for image processing and file handling
import os
import numpy as np
from PIL import Image
from pathlib import Path

In [2]:
# Define input and output paths, grid size, and resize factor
input_folder = Path("recon")  # Relative path to the recon/ folder
output_image_path = Path("output_grid.png")  # Output in the main folder

# Define grid size (rows, columns) and resize factor for images
grid_size = (23, 23)  # Adjust grid size as needed
resize_factor = 0.4  # Resize factor to reduce image dimensions

In [3]:
# Check if the input folder exists and retrieve a list of .tif files
if not input_folder.exists():
    print(f"Input folder '{input_folder}' does not exist.")
else:
    # Get a sorted list of all .tif files in the input folder
    image_paths = sorted(input_folder.glob("*.tif"))
    
    # Limit the number of images to the required grid size
    required_images = grid_size[0] * grid_size[1]
    image_paths = image_paths[:required_images]
    
    print(f"Number of images to load: {len(image_paths)}")

Number of images to load: 529


In [4]:
# Load images into memory and determine the global minimum and maximum pixel values
global_min = float('inf')
global_max = float('-inf')
images = []

for file_path in image_paths:
    # Open image and convert to 16-bit grayscale if necessary
    img = Image.open(file_path)
    if img.mode != 'I;16':
        img = img.convert('I;16')
    
    # Convert image to numpy array and store it
    img_array = np.array(img)
    images.append(img_array)
    
    # Update global minimum and maximum values
    min_val = img_array.min()
    max_val = img_array.max()
    if min_val < global_min:
        global_min = min_val
    if max_val > global_max:
        global_max = max_val

print(f"Global min: {global_min}, Global max: {global_max}")

Global min: 8334, Global max: 55339


In [5]:
# Normalize image pixel values to the range [0, 255] and convert to uint8
normalized_images = []

for img_array in images:
    # Normalize to [0, 1] based on global min and max
    img_normalized = (img_array - global_min) / (global_max - global_min)
    # Scale to [0, 255] and convert to 8-bit unsigned integers
    img_8bit = (img_normalized * 255).astype(np.uint8)
    normalized_images.append(img_8bit)

In [6]:
# Resize images if a resize factor other than 1.0 is specified
if resize_factor != 1.0:
    resized_images = []
    for img_array in normalized_images:
        # Convert numpy array back to PIL Image for resizing
        img = Image.fromarray(img_array)
        # Calculate new dimensions based on the resize factor
        new_size = (int(img.width * resize_factor), int(img.height * resize_factor))
        # Resize image using bicubic interpolation
        img_resized = img.resize(new_size, Image.BICUBIC)
        # Convert resized image back to numpy array and store it
        resized_images.append(np.array(img_resized))
else:
    resized_images = normalized_images

In [7]:
# Stack all image arrays into a single 3D numpy array for grid creation
slices = np.stack(resized_images, axis=0)
print(f"Stacked image array shape: {slices.shape}")

Stacked image array shape: (529, 742, 742)


In [8]:
# Create a grid image by arranging the individual images according to the grid size
num_slices = slices.shape[0]
slice_height, slice_width = slices.shape[1:3]

# Create a new blank image to hold the grid
grid_image = Image.new('L', (slice_width * grid_size[1], slice_height * grid_size[0]))

# Paste each image slice into the appropriate position in the grid
for i in range(num_slices):
    row = i // grid_size[1]
    col = i % grid_size[1]
    
    # Break the loop if the grid is filled
    if row >= grid_size[0]:
        break
    
    # Get the image slice and paste it into the grid image
    slice_data = slices[i, :, :]
    slice_image = Image.fromarray(slice_data)
    grid_image.paste(slice_image, (col * slice_width, row * slice_height))

In [9]:
# Save the final grid image to the specified output path and display it
grid_image.save(output_image_path)
print(f"Grid image saved to {output_image_path}")

# Optionally display the grid image (may not work in all environments)
grid_image.show()

Grid image saved to output_grid.png
